In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
data = pd.read_csv(r'C:/Users/Jordan/Desktop/jupyter notebook/Dissertation Code/data.csv')
data.head()

C:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (6,30) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,file,year,manufacturer,model,description,euro_standard,tax_band,transmission,transmission_type,engine_capacity,...,nox_emissions,thc_nox_emissions,particulates_emissions,fuel_cost_12000_miles,fuel_cost_6000_miles,standard_12_months,standard_6_months,first_year_12_months,first_year_6_months,date_of_change
0,DatapartC_july2000.csv,2000,Alfa Romeo,145 Range,1.6 Twin Spark 16v,2,NaN,M5,Manual,1598.0,...,NaN,NaN,NaN,NaN,618.0,NaN,NaN,NaN,NaN,NaN
1,DatapartC_july2000.csv,2000,Alfa Romeo,145 Range,1.8 Twin Spark 16v,2,NaN,M5,Manual,1747.0,...,NaN,NaN,NaN,NaN,633.0,NaN,NaN,NaN,NaN,NaN
2,DatapartC_july2000.csv,2000,Alfa Romeo,145 Range,Cloverleaf,2,NaN,M5,Manual,1970.0,...,NaN,NaN,NaN,NaN,663.0,NaN,NaN,NaN,NaN,NaN
3,DatapartC_july2000.csv,2000,Alfa Romeo,146 Range,1.6 Twin Spark 16v,2,NaN,M5,Manual,1598.0,...,NaN,NaN,NaN,NaN,626.0,NaN,NaN,NaN,NaN,NaN
4,DatapartC_july2000.csv,2000,Alfa Romeo,146 Range,1.8 Twin Spark 16v,2,NaN,M5,Manual,1747.0,...,NaN,NaN,NaN,NaN,633.0,NaN,NaN,NaN,NaN,NaN


In [3]:
#multiplying 6k miles column by 2 to get more data in the 12k mile column
data['fuel_cost_6000_miles'] = data['fuel_cost_6000_miles'] * 2

In [4]:
#Combining 6k and 12k miles column together
#Ensuring the null/NaN values are not included e.g. NaN1230.0 it is represented as just the number using this.
data['fuel_cost_6000_miles'] = data['fuel_cost_12000_miles'].fillna(data['fuel_cost_6000_miles'])

In [5]:
#Renaming column
data = data.rename(columns={'fuel_cost_6000_miles': 'fuel_12k_miles',
                       'co_emissions': 'co'})

In [6]:
#Droppping unnecessary columns
data = data.drop(columns= ['file', 'tax_band', 'transmission', 'noise_level', 'combined_metric', 'urban_imperial',
                       'extra_urban_imperial', 'combined_imperial', 'noise_level', 'thc_emissions', 'nox_emissions',
                       'thc_nox_emissions', 'particulates_emissions', 'standard_12_months', 'standard_6_months',
                       'first_year_12_months', 'first_year_6_months', 'date_of_change', 'fuel_cost_12000_miles'])
data.head()

,year,manufacturer,model,description,euro_standard,transmission_type,engine_capacity,fuel_type,urban_metric,extra_urban_metric,co2,co,fuel_12k_miles
0,2000,Alfa Romeo,145 Range,1.6 Twin Spark 16v,2,Manual,1598.0,Petrol,11.1,6.5,195,980.0,1236.0
1,2000,Alfa Romeo,145 Range,1.8 Twin Spark 16v,2,Manual,1747.0,Petrol,11.5,6.6,200,1105.0,1266.0
2,2000,Alfa Romeo,145 Range,Cloverleaf,2,Manual,1970.0,Petrol,12.4,6.6,210,1103.0,1326.0
3,2000,Alfa Romeo,146 Range,1.6 Twin Spark 16v,2,Manual,1598.0,Petrol,11.3,6.6,195,980.0,1252.0
4,2000,Alfa Romeo,146 Range,1.8 Twin Spark 16v,2,Manual,1747.0,Petrol,11.5,6.6,200,1105.0,1266.0


In [7]:
#Showing null/NaN records in each column
data.isnull().sum()

year                    0
manufacturer            0
model                   0
description             0
euro_standard           0
transmission_type     341
engine_capacity         7
fuel_type               0
urban_metric           13
extra_urban_metric     13
co2                     0
co                    319
fuel_12k_miles        175
dtype: int64

In [8]:
#Dropping null/NaN records
data.dropna(subset=['transmission_type','urban_metric','extra_urban_metric','co','fuel_12k_miles'], inplace=True)

In [9]:
#Showing null/NaN records deleted
data.isnull().sum()

year                  0
manufacturer          0
model                 0
description           0
euro_standard         0
transmission_type     0
engine_capacity       0
fuel_type             0
urban_metric          0
extra_urban_metric    0
co2                   0
co                    0
fuel_12k_miles        0
dtype: int64

In [10]:
#removing special characters such as spaces, ()&/- etc... from fuel type
data['fuel_type'] = data['fuel_type'].map(lambda x: re.sub(r'\W+', '', x))

In [11]:
#Converting all petrol alternatives to petrol
data['fuel_type'] = data['fuel_type'].str.replace('LPG','Petrol')
data['fuel_type'] = data['fuel_type'].str.replace('LPGPetrol','Petrol')
data['fuel_type'] = data['fuel_type'].str.replace('PetrolE85','Petrol')
data['fuel_type'] = data['fuel_type'].str.replace('PetrolE85Flex Fuel','Petrol')
data['fuel_type'] = data['fuel_type'].str.replace('PetrolFlexFuel','Petrol')
data['fuel_type'] = data['fuel_type'].str.replace('PetrolPetrol','Petrol')

In [12]:
#Converting all Hybrid petrol alternatives (Electric + Petrol)
data['fuel_type'] = data['fuel_type'].str.replace('PetrolElectric','Hybrid Petrol')
data['fuel_type'] = data['fuel_type'].str.replace('ElectricityPetrol','Hybrid Petrol')
data['fuel_type'] = data['fuel_type'].str.replace('PetrolHybrid','Hybrid Petrol')

In [13]:
#Converting all Hybrid Diesel alternatives (Electric + Diesel)
data['fuel_type'] = data['fuel_type'].str.replace('DieselElectric','Hybrid Diesel')
data['fuel_type'] = data['fuel_type'].str.replace('ElectricityDiesel','Hybrid Diesel')

In [14]:
data['transmission'] = data['transmission_type'].factorize()[0]
#Transmission:
# 0 = Manual
# 1 = Automatic

data['fueltype'] = data['fuel_type'].factorize()[0]
#Fuel Type
# 0 = Petrol
# 1 = Diesel
# 2 = Hybrid Petrol
# 3 = CNG
# 4 = Hybrid Diesel

In [15]:
#cars are shown as 1.6 engine as opposed to 1598
#Rounding them to nearest hundred allows for easier computation
data['engine_capacity'] = round(data['engine_capacity'],-2)

In [16]:
data.head()

,year,manufacturer,model,description,euro_standard,transmission_type,engine_capacity,fuel_type,urban_metric,extra_urban_metric,co2,co,fuel_12k_miles,transmission,fueltype
0,2000,Alfa Romeo,145 Range,1.6 Twin Spark 16v,2,Manual,1600.0,Petrol,11.1,6.5,195,980.0,1236.0,0,0
1,2000,Alfa Romeo,145 Range,1.8 Twin Spark 16v,2,Manual,1700.0,Petrol,11.5,6.6,200,1105.0,1266.0,0,0
2,2000,Alfa Romeo,145 Range,Cloverleaf,2,Manual,2000.0,Petrol,12.4,6.6,210,1103.0,1326.0,0,0
3,2000,Alfa Romeo,146 Range,1.6 Twin Spark 16v,2,Manual,1600.0,Petrol,11.3,6.6,195,980.0,1252.0,0,0
4,2000,Alfa Romeo,146 Range,1.8 Twin Spark 16v,2,Manual,1700.0,Petrol,11.5,6.6,200,1105.0,1266.0,0,0


In [17]:
#Changing data types from float to integer
data['engine_capacity'] = data['engine_capacity'].astype(int)
data['co'] = data['co'].astype(int)
data['fuel_12k_miles'] = data['fuel_12k_miles'].astype(int)

In [18]:
data = data.drop(columns= ['transmission_type', 'fuel_type'])

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44697 entries, 0 to 45177
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   year                44697 non-null  int64  
 1   manufacturer        44697 non-null  object 
 2   model               44697 non-null  object 
 3   description         44697 non-null  object 
 4   euro_standard       44697 non-null  int64  
 5   engine_capacity     44697 non-null  int32  
 6   urban_metric        44697 non-null  float64
 7   extra_urban_metric  44697 non-null  float64
 8   co2                 44697 non-null  int64  
 9   co                  44697 non-null  int32  
 10  fuel_12k_miles      44697 non-null  int32  
 11  transmission        44697 non-null  int64  
 12  fueltype            44697 non-null  int64  
dtypes: float64(2), int32(3), int64(5), object(3)
memory usage: 4.3+ MB


In [20]:
data.head()

,year,manufacturer,model,description,euro_standard,engine_capacity,urban_metric,extra_urban_metric,co2,co,fuel_12k_miles,transmission,fueltype
0,2000,Alfa Romeo,145 Range,1.6 Twin Spark 16v,2,1600,11.1,6.5,195,980,1236,0,0
1,2000,Alfa Romeo,145 Range,1.8 Twin Spark 16v,2,1700,11.5,6.6,200,1105,1266,0,0
2,2000,Alfa Romeo,145 Range,Cloverleaf,2,2000,12.4,6.6,210,1103,1326,0,0
3,2000,Alfa Romeo,146 Range,1.6 Twin Spark 16v,2,1600,11.3,6.6,195,980,1252,0,0
4,2000,Alfa Romeo,146 Range,1.8 Twin Spark 16v,2,1700,11.5,6.6,200,1105,1266,0,0


In [21]:
data.to_csv('df.csv')